In [2]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue

# from src import data 
# from src import analyzer


import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn import linear_model
# import cufflinks as cf
# cf.go_offline()

In [3]:
ov_chg = pd.read_csv("/home/hengtian/kate/chg/ovngt_chg.csv", index_col = 0)

In [7]:
ov_chg = ov_chg.drop('idx',axis = 1)

In [4]:
am_chg = pd.read_csv("/home/hengtian/kate/chg/am_chg.csv", index_col = 0)

In [5]:
pm_chg = pd.read_csv("/home/hengtian/kate/chg/pm_chg.csv", index_col = 0)

In [11]:
l=len(ov_chg)

In [23]:
def apm_w(stock,q):
    i = 20
    diff = np.full(l,np.nan)
    am = am_chg[stock]
    pm = pm_chg[stock]
    ovngt =ov_chg[stock]
    while i<l:
        s_ov = ovngt.iloc[i-20:i]
        s_am =am.iloc[i-20:i]
        s_pm = pm.iloc[i-20:i]
        if len(s_am.dropna())<20 or len(s_pm.dropna())<20:
            i+=1
        else:
            am_sum = s_am.sum()
            ov_sum = s_ov.sum()
            pm_sum = s_pm.sum()
            diff[i] =am_sum-pm_sum
            i+=1
    return q.put({stock:diff})

In [24]:
q = multiprocessing.Manager().Queue()
param = []
col =ov_chg.columns[:-1]
for stock in col:
    param.append((stock,q))
# print(param)
#task
with multiprocessing.Pool(processes =4) as pool:
    pool.starmap(apm_w,param)

#extract
elements = []
while q.qsize():
    elements.append(q.get())

In [25]:
d = pd.DataFrame(np.empty((len(ov_chg),len(col)),dtype=object),columns = col, index = ov_chg.index)
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    val = list(elements[i][stock])
    d[stock]=val

In [26]:
d.to_csv("apm_avp.csv")